In [11]:
p = 93556643250795678718734474880013829509320385402690660619699653921022012489089
a = 66001598144012865876674115570268990806314506711104521036747533612798434904785
b = 25255205054024371783896605039267101837972419055969636393425590261926131199028
x = 1663255323649316187237502679180020234514169346773977936323045878120391437837
y = 84006750294604478804216697619398950537391904455312244388797990516611936533488
E = EllipticCurve(GF(p), [a, b])
G = E(x, y)

In [33]:
G_order = 93556643250795678718734474880013829509848977747379500929457472016889767786158
G2_order = 46778321625397839359367237440006914754924488873689750464728736008444883893079
G3_order = 93556643250795678718734474880013829509848977747379500929457472016889767786158

def elliptic_legendre(point):
    return (point * (G_order // 2))[0] == 0

print(elliptic_legendre(G * 2))
print(elliptic_legendre(G * 3))

True
False


In [45]:
from Crypto.Util.number import long_to_bytes
from pwn import *

def send_to_encrypt(m):
    m = hex(m)[2:]
    if len(m) % 2 == 1:
        m = m.zfill(len(m) + 1)

    io.sendlineafter(b">>> ", b"1")
    io.sendlineafter(b"Plaintext (hex): ", m.encode())

    return int(io.recvline().decode(), 16)

def recover_n(cs):
    kn1 = cs[0]**5 - cs[1]
    kn2 = cs[0]**8 - cs[2]
    kn3 = cs[0]**21 - cs[3]

    n = gcd(gcd(kn1, kn2), kn3)
    for i in range(2, 10000):
        if n % i == 0:
            n //= i
    return n

def elliptic_legendre(point):
    return (point * (G_order // 2))[0] == 0

def send_to_oracle(c):
    c = hex(c)[2:]
    if len(c) % 2 == 1:
        c = c.zfill(len(c) + 1)

    io.sendlineafter(b">>> ", b"2")
    io.sendlineafter(b"Ciphertext (hex): ", c.encode())

    point = E([int(x) for x in io.recvline().decode()[1:-2].split(", ")])
    return not elliptic_legendre(point)

io = process(["python3", "/tmp/challenges/empoleon/chall.py"])
context.log_level = "debug"

io.sendlineafter(b">>> ", b"3")
c_flag = int(io.recvline().decode(), 16)
print("c_flag =", c_flag)

cs = [send_to_encrypt(m) for m in [2, 8, 16, 128]]
n = recover_n(cs)
print("n =", n)

e = 0x10001
upper_limit = n
lower_limit = 0

flag = ""
i = 1
context.log_level = "info"
while i <= n.bit_length() + 10:
    print(i)
    if not send_to_oracle(c_flag * pow(2**i, e, n) % n):
        upper_limit = (upper_limit + lower_limit)//2
    else:
        lower_limit = (lower_limit + upper_limit)//2
    i += 1

print(long_to_bytes(lower_limit))

[x] Starting local process '/home/sage/sage/local/var/lib/sage/venv-python3.12.5/bin/python3'
[+] Starting local process '/home/sage/sage/local/var/lib/sage/venv-python3.12.5/bin/python3': pid 645
[DEBUG] Received 0x1b bytes:
    b'Empoleon challenges you...\n'
[DEBUG] Received 0x22 bytes:
    b'1. Encrypt\n'
    b'2. Decrypt\n'
    b'3. Flag\n'
    b'>>> '
[DEBUG] Sent 0x2 bytes:
    b'3\n'
[DEBUG] Received 0x123 bytes:
    b'908b7060a0458fc3cda61ccf71a6d02f9b3396e9cd056f69a532216c2c85f046f438a488ccd1c9a3df5412eb0f65c7432e7f40fb44efe67c65c087a89b20ab7b6d81254ee565a8eca6a4425425e2f3d01885d3acd569d0d48fbf0af22030881357fc5288eddacb7dffc1d144f8a3b73f021a4bf4b9e82283c9157eb91b6e62f0\n'
    b'1. Encrypt\n'
    b'2. Decrypt\n'
    b'3. Flag\n'
    b'>>> '
c_flag = 101502728592788061664949753978250472623014709995216528958412164087579046669531809474567702882794193795684099886136364423785209553507498407549045812872142212965329011005834689641989569635201438730807486021308943534782956659774048148

In [ ]:
from Crypto.Util.number import long_to_bytes
from pwn import *

def send_to_encrypt(m):
    m = hex(m)[2:]
    if len(m) % 2 == 1:
        m = m.zfill(len(m) + 1)

    io.sendlineafter(b">>> ", b"1")
    io.sendlineafter(b"Plaintext (hex): ", m.encode())

    return int(io.recvline().decode(), 16)

def recover_n(cs):
    kn1 = cs[0]**5 - cs[1]
    kn2 = cs[0]**8 - cs[2]
    kn3 = cs[0]**21 - cs[3]

    n = gcd(gcd(kn1, kn2), kn3)
    for i in range(2, 10000):
        if n % i == 0:
            n //= i
    return n

def elliptic_legendre(point):
    return (point * (G_order // 2))[0] == 0

def send_to_oracle(c):
    c = hex(c)[2:]
    if len(c) % 2 == 1:
        c = c.zfill(len(c) + 1)

    io.sendlineafter(b">>> ", b"2")
    io.sendlineafter(b"Ciphertext (hex): ", c.encode())

    point = E([int(x) for x in io.recvline().decode()[1:-2].split(", ")])
    return not elliptic_legendre(point)

io = remote("ctf.arkavidia.com", 8042)
context.log_level = "debug"

io.sendlineafter(b">>> ", b"3")
c_flag = int(io.recvline().decode(), 16)
print("c_flag =", c_flag)

cs = [send_to_encrypt(m) for m in [2, 8, 16, 128]]
n = recover_n(cs)
print("n =", n)

e = 0x10001
upper_limit = n
lower_limit = 0

flag = ""
i = 1
context.log_level = "info"
while i <= n.bit_length() + 10:
    print(i)
    if not send_to_oracle(c_flag * pow(2**i, e, n) % n):
        upper_limit = (upper_limit + lower_limit)//2
    else:
        lower_limit = (lower_limit + upper_limit)//2
    i += 1

print(long_to_bytes(lower_limit))
# ARKAV{the_hardest_part_of_computer_science_is_naming_variables_challs_and_flags_9e2bf30ae454ab36}

[x] Opening connection to ctf.arkavidia.com on port 8042


[x] Opening connection to ctf.arkavidia.com on port 8042: Trying 20.188.114.191
[+] Opening connection to ctf.arkavidia.com on port 8042: Done
[DEBUG] Received 0x39 bytes:
    b'Another RSA machine...\n'
    b'1. Encrypt\n'
    b'2. Decrypt\n'
    b'3. Flag\n'
    b'>>> '
[DEBUG] Sent 0x2 bytes:
    b'3\n'
[DEBUG] Received 0x123 bytes:
    b'662f0ea45cfa3636e782cdbc5c51822f3d61c9e912ca3d262f2f675417739e5d92019bb01c6bfa348fc76c37db12aff06a0a51265377571ac7fd7571781aa459d891774d8c5ff5629d3d7604fb6ea7659139ef274efe2e8fe6b068f81f3ea755d7e28799dac0976709b35062267482bcbc2923219b7f027419c570752744c44b\n'
    b'1. Encrypt\n'
    b'2. Decrypt\n'
    b'3. Flag\n'
    b'>>> '
c_flag = 71755916648545804386591428232901578329716393688834686127144006772593099997439591499937168766369038799466148589312884497002632820596663914072701652065368642358957695788724266354585220213389628597953916622397345214788929524465375857292937971113993687290561910175157049593852534946957388575617228619201695237195
[DEBUG] S